In [7]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [8]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=r"C:\Users\nisal\Downloads\IoT\ClassificationModel\train_classifier_model.tflite")
interpreter.allocate_tensors()

In [9]:
# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [10]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((96, 96))
    img = np.array(img)

    # Check if model expects quantized input (uint8) or float32
    if input_details[0]['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details[0]['quantization']
        img = img / 255.0  # normalize
        img = img / input_scale + input_zero_point
        img = np.clip(img, 0, 255).astype(np.uint8)
    else:
        img = img.astype(np.float32) / 255.0

    return np.expand_dims(img, axis=0)  # add batch dimension

In [11]:
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = output_data[0][0]

    if output_details[0]['dtype'] == np.uint8:
        output_scale, output_zero_point = output_details[0]['quantization']
        prediction = (prediction - output_zero_point) * output_scale

    print(f"Prediction: {prediction:.4f} → {'Train' if prediction > 0.5 else 'Not a Train'}")

In [12]:
test_folder = r"C:\Users\nisal\Downloads\IoT\input_data"
for fname in os.listdir(test_folder):
    if fname.endswith(".jpg") or fname.endswith(".png"):
        print(f"\nImage: {fname}")
        classify_image(os.path.join(test_folder, fname))


Image: test1.jpg
Prediction: 0.3398 → Not a Train

Image: test2.jpg
Prediction: 0.9570 → Train

Image: test3.jpg
Prediction: 0.7148 → Train

Image: test4.jpg
Prediction: 0.8594 → Train

Image: test5.jpg
Prediction: 0.8984 → Train

Image: test6.jpg
Prediction: 0.9219 → Train

Image: train1.jpg
Prediction: 0.3750 → Not a Train
